# Tasks

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

# Imports

In [1]:
# Standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# New Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Install tmdbsimple 
!pip install tqdm

## Login Information

In [11]:
import json
with open('/Users/jasontracey/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)

yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

# Function

In [12]:
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists    
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            

    ## If it does NOT exist:
    else:
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

# Set Search Parameters

In [13]:
# set our API call parameters 
LOCATION = 'San Diego, CA'
TERM = 'Sushi'

In [16]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_San_Diego_sushi.json"
JSON_FILE

'Data/results_in_progress_San_Diego_sushi.json'

In [17]:
create_json_file(JSON_FILE,  delete_if_exists=False)

[i] Data/results_in_progress_San_Diego_sushi.json not found. Saving empty list to new file.


In [18]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [19]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
## How many results total?
total_results = results['total']
total_results

1600

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [22]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

80

In [23]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [24]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/80 [00:00<?, ?it/s]

In [26]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_San_Diego_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_San_Diego_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


80

In [31]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.1)

  0%|          | 0/80 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [32]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,3S6VtYgcXL8lRic0iV5GYQ,sushi-ota-san-diego,Sushi Ota,https://s3-media3.fl.yelpcdn.com/bphoto/rWQ8VS...,False,https://www.yelp.com/biz/sushi-ota-san-diego?a...,4386,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 32.8036636, 'longitude': -117.217...",[delivery],$$$,"{'address1': '4529 Mission Bay Dr', 'address2'...",+18588808778,(858) 880-8778,11596.015168
1,O1NFJA0OI0-N77JzP7qlfQ,azuki-sushi-san-diego-2,Azuki Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/OhZXVg...,False,https://www.yelp.com/biz/azuki-sushi-san-diego...,2866,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 32.7296447753906, 'longitude': -1...","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '2321 5th Ave', 'address2': '', '...",+16192384760,(619) 238-4760,12080.290756
2,_8-J0_Q4YmHhWTGEx1bCew,kaki-sushi-san-diego,Kaki Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/8C1FWN...,False,https://www.yelp.com/biz/kaki-sushi-san-diego?...,187,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 32.83017, 'longitude': -117.10431}","[delivery, pickup]",$$,"{'address1': '10428 Clairemont Mesa Blvd', 'ad...",+18585712012,(858) 571-2012,1162.353505
3,yp7SqJJuN6xedIWjz_MH1Q,sushi-diner-2-san-diego,Sushi Diner 2,https://s3-media1.fl.yelpcdn.com/bphoto/1JBW6K...,False,https://www.yelp.com/biz/sushi-diner-2-san-die...,461,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.0,"{'latitude': 32.791911, 'longitude': -117.100473}",[delivery],$$,"{'address1': '10330 Friars Rd', 'address2': 'S...",+16192816448,(619) 281-6448,3553.159094
4,kUv1KwH-N3ATlRfzOX7nCg,kura-revolving-sushi-bar-san-diego-2,Kura Revolving Sushi Bar,https://s3-media1.fl.yelpcdn.com/bphoto/KCWuH6...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,2393,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 32.8238827, 'longitude': -117.15469}",[delivery],$$,"{'address1': '4609 Convoy St', 'address2': 'St...",+18587154605,(858) 715-4605,5574.702568


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,7ffENycJAzU8w51xn5sWlQ,cowboy-star-san-diego,Cowboy Star,https://s3-media1.fl.yelpcdn.com/bphoto/ueYHzE...,False,https://www.yelp.com/biz/cowboy-star-san-diego...,1852,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 32.71220369054662, 'longitude': -...",[delivery],$$$,"{'address1': '640 10th Ave', 'address2': '', '...",+16194505880,(619) 450-5880,13618.833044
996,We1K46Xs7GDDkzcAkC9uGg,sol-agave-san-clemente-san-clemente,Sol Agave - San Clemente,https://s3-media3.fl.yelpcdn.com/bphoto/MNxdLs...,False,https://www.yelp.com/biz/sol-agave-san-clement...,442,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 33.426863, 'longitude': -117.613242}","[pickup, delivery]",$$,"{'address1': '111 Avenida Del Mar', 'address2'...",+19493122210,(949) 312-2210,82644.198466
997,4e9iy-2OVNL3eh_aiCVJSw,fig-tree-cafe-san-diego-7,Fig Tree Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/mKW99Y...,False,https://www.yelp.com/biz/fig-tree-cafe-san-die...,201,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 32.712376, 'longitude': -117.1590...","[pickup, delivery]",$$,"{'address1': '695 Sixth Ave', 'address2': None...",+16193494890,(619) 349-4890,13727.532195
998,B1NfUF_EsOyFahUulUgvtw,the-spot-san-diego-5,The Spot,https://s3-media3.fl.yelpcdn.com/bphoto/MHpYen...,False,https://www.yelp.com/biz/the-spot-san-diego-5?...,3,"[{'alias': 'foodtrucks', 'title': 'Food Trucks'}]",5.0,"{'latitude': 32.753795960876886, 'longitude': ...",[],NaN,"{'address1': '3742 Midway Dr', 'address2': '',...",,,13796.170118
999,gBShJ0bHyXuQXhbKfEhZfg,uptown-tavern-san-diego,Uptown Tavern,https://s3-media3.fl.yelpcdn.com/bphoto/L2_AxM...,False,https://www.yelp.com/biz/uptown-tavern-san-die...,646,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 32.7486064912939, 'longitude': -1...",[delivery],$$,"{'address1': '1236 University Ave', 'address2'...",+16192412710,(619) 241-2710,9921.803178


In [33]:
# check for duplicate results
final_df.duplicated(subset = 'id').sum()

0

In [34]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_San_Diego_sushi.csv.gz', compression='gzip',index=False)